### Fine-tune Llama2 using QLoRA, Bits and Bytes and PEFT
***

- **[QLoRA](https://arxiv.org/pdf/2305.14314.pdf): Quantized Low Rank Adapters** - a method for fine-tuning LLMs that uses a small number of quantized, updateable parameters to limit the complexity of training. This technique also allows those small sets of parameters to be added efficiently into the model itself, which means you can do fine-tuning on lots of data sets, potentially, and swap these "adapters" into your model when necessary.

- **[Bits and Bytes](https://github.com/TimDettmers/bitsandbytes)**: Excellent package by Tim Dettmers et al., which provides a lightweight wrapper around custom CUDA functions that make LLMs go faster - optimizers, matrix mults and quantization.

- **[PEFT](https://github.com/huggingface/peft): Parameter Efficient Fine-tuning**: Huggingface library that enables a number of PEFT methods, which again make it less expensive to fine-tune LLMs. These methods enable efficient adaptation of pre-trained language models (PLMs) to various downstream applications without fine-tuning all the model's parameters

## Installs and imports

In [ ]:
%pip install -qqq bitsandbytes==0.41.1
%pip install -qqq loralib==0.1.1
%pip install -qqq peft==0.5.0

In [2]:
import os
import re
import random

from tqdm import tqdm
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from wordcloud import WordCloud

import transformers
from transformers import (
    AutoModelForCausalLM,      
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)

import torch
from torch.utils.data import DataLoader, Dataset

import warnings
warnings.filterwarnings("ignore")

tqdm.pandas()
sns.set_style("dark")
plt.rcParams["figure.figsize"] = (20,8)
plt.rcParams["font.size"] = 14

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def seed_everything(seed=None):
    
    max_seed_value = np.iinfo(np.uint32).max
    min_seed_value = np.iinfo(np.uint32).min

    if (seed is None) or not (min_seed_value <= seed <= max_seed_value):
        seed = random.randint(np.iinfo(np.uint32).min, np.iinfo(np.uint32).max)
        
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    return seed

seed_everything(42)

42

***
### Load model

In [4]:
%%time

modelx = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"
tokenizer = AutoTokenizer.from_pretrained(modelx)

model = AutoModelForCausalLM.from_pretrained(
    modelx,
    torch_dtype=torch.float16,
    device_map='auto'
)

# Calculate the size of the model in bytes
model_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())

# Convert to megabytes (MB)
model_size_megabytes = model_size_bytes / (1024 * 1024)

print(f"Model size: {model_size_megabytes:.2f} MB")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model size: 12852.51 MB
CPU times: user 6.54 s, sys: 9.67 s, total: 16.2 s
Wall time: 3min 23s


In [7]:
!nvidia-smi --query-gpu=memory.used --format=csv,noheader,nounits

13715


In [8]:
del model
# del qmodel
torch.cuda.empty_cache()

***
### Prepare and load model
Use Bits and Bytes with 4-bit quantization configuration, load the weights in 4-bit format, using a normal float 4 with double quantization to improve QLoRA's resolution. The weights are converted back to bfloat16 for weight updates.

In [9]:
%%time

bnb_config = BitsAndBytesConfig(
    load_in_8bit=False,
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

qmodel = AutoModelForCausalLM.from_pretrained(
    modelx,
    use_safetensors=True,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto",
)

# Calculate the size of the model in bytes
model_size_bytes = sum(p.numel() * p.element_size() for p in qmodel.parameters())

# Convert to megabytes (MB)
model_size_megabytes = model_size_bytes / (1024 * 1024)

print(f"Model size: {model_size_megabytes:.2f} MB")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model size: 3588.51 MB
CPU times: user 5.22 s, sys: 10.9 s, total: 16.1 s
Wall time: 2min 33s


In [10]:
!nvidia-smi --query-gpu=memory.used --format=csv,noheader,nounits

5011
